In [1]:
%load_ext autoreload
%autoreload 2
from main import do
from main import do_v2
import numpy as np
from sklearn.model_selection import train_test_split
from preprocess import contents2count, contents2count_v2
from sklearn.metrics.pairwise import cosine_similarity
from preprocess import AE2
from tqdm import tqdm
from sklearn.metrics import f1_score
import pandas as pd 
from collections import Counter
import random
from glob import glob
import os


c:\Users\user\anaconda3\envs\deepcase\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def hex2ascii(x):
    output = ''
    for i in range(0,len(x),2):
        tmp = str(x[i:i+2])
        output+=chr(int(tmp,16))
    return output

In [3]:
def read_opendata(path):
    datas_ = []
    for paths in glob(os.path.join(path,'*.txt')):
        datas = []
        with open(paths) as f:
            event=''
            tmp1=''
            tmp2=''
            for i in f.readlines():
                if (i[:3]=='GET' )| (i[:4]=='POST'):
                    datas.append(event)
                    event = i
                else:
                    event+=i
        datas_.append(datas[1:])



    labels = list(map(lambda x: np.zeros(len(x)), datas_))
    labels[0] +=1

    del datas
    tr1,te1 =  train_test_split(datas_[0],test_size=0.2,shuffle=True,random_state=666)
    tr2,te2 =  train_test_split(datas_[1],test_size=0.2,shuffle=True,random_state=666)

    train = tr1 + tr2
    train_label = np.array([0]*len(tr1) + [1]*len(tr2))
    test = te1 + te2
    test_label = np.array([0]*len(te1) + [1]*len(te2))

    train = list(map(lambda x: "\n".join([x.splitlines()[0],x.splitlines()[-2]]) , train))
    test_ = list(map(lambda x: "\n".join([x.splitlines()[0],x.splitlines()[-2]]) , test))
    return train, test_ ,train_label,test_label

In [4]:
def find_centroid(res,test,th):
    
    cluster_center = {}
    for i in res[1]:
        cluster_center[i] = np.mean(res[1][i],axis=0)

    ress = []
    keys = list(cluster_center.keys())
    cluster_center_vectors = np.array(list(cluster_center.values()))

    for ev in tqdm(test):
        ev = ev.reshape(1,-1)
        sim = cosine_similarity(ev,cluster_center_vectors)
        sim_max = np.max(sim)
        if sim_max<th:
            ress.append(-1)
        else:
            ress.append(keys[np.argmax(sim)])

    return np.array(ress)

In [11]:
def random_pick(x,n):
    p = np.random.choice(int(x[0]+x[1]-1), n,replace=False)
    return np.array(list(map(lambda y: 0 if x[0]>y  else 1 , p)))

In [6]:
def clustering_labeling(res,train_label,n):
    df = pd.DataFrame([res,train_label])
    df = df.transpose()
    df.columns = ['pred','label']
    values = df.groupby('pred').value_counts().reset_index()
    card = df.groupby('pred').value_counts().reset_index().groupby('pred').nunique().reset_index()
    card.columns = ['pred','label','card']
    card[0]=0
    card[1]=0

    for _,i in values.iterrows():
        card.loc[card.pred == i['pred'],i.label] = i[0]

    card['ratio'] = card[1]/(card[0] + card[1])

    card['random_pick'] = card.apply(random_pick,axis=1,n=n)
    cluster_label = card.set_index('pred').random_pick.to_dict()
    return card,cluster_label
 

In [17]:
def testing(card,pred,cluster_label):
    pred_ = np.array(list(map(lambda x: cluster_label[x],pred)))
    return pred_

In [8]:

th = 0.95
n = 1

train, test_ ,train_label,test_label =  read_opendata('./dataset/')

res = do_v2(train,512,4,th,th,3)

test = res[-1].transform(test_)

cluster_center = find_centroid(res,test,th)

pred = find_centroid(res,test,th)

card,cluster_label = clustering_labeling(res[0],train_label,n)

pred_ = testing(card,pred,cluster_label)
coverage_index = pred!=-1

f1  = f1_score(test_label[coverage_index],pred_[coverage_index])

coverage = coverage_index.sum()/coverage_index.shape[0]

card,cluster_label = clustering_labeling(res[0],train_label,n)

pred_ = testing(card,pred,cluster_label)
coverage_index = pred!=-1

f1  = f1_score(test_label[coverage_index],pred_[coverage_index])

coverage = coverage_index.sum()/coverage_index.shape[0]

12134it [05:37, 35.95it/s]
12134it [05:40, 35.59it/s]


ValueError: a must be 1-dimensional or an integer